In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3

# Load the InceptionV3 model with pre-trained weights
model = InceptionV3(weights='imagenet', include_top=False) 


In [2]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import InceptionV3

# Load the base Inception model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(229, 229, 3))

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add new layers on top of the base model
model = models.Sequential()
model.add(base_model)

# Add Global Average Pooling instead of Flatten
model.add(layers.GlobalAveragePooling2D())

# Adding Dense layers
model.add(layers.Dense(512, activation='relu'))  # Adding a dense layer
model.add(layers.Dropout(0.5))  # Dropout for regularization
model.add(layers.Dense(256, activation='relu'))  # Another dense layer
model.add(layers.Dense(7, activation='softmax'))  # Adjust num_classes to your use case

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assign the model to Dmodel
Dmodel = model


In [3]:

def triplet_loss(y_true, y_pred, alpha=0.2):
  
    anchor, positive, negative = y_pred
    
    # Ensure that the inputs are tensors
    anchor = tf.convert_to_tensor(anchor, dtype=tf.float32)
    positive = tf.convert_to_tensor(positive, dtype=tf.float32)
    negative = tf.convert_to_tensor(negative, dtype=tf.float32)
    
    # Step 1: Compute the (encoding) distance between the anchor and the positive
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    
    # Step 2: Compute the (encoding) distance between the anchor and the negative
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    
    # Step 3: Subtract the two previous distances and add alpha
    basic_loss = pos_dist - neg_dist + alpha
    
    # Step 4: Take the maximum of basic_loss and 0.0, then compute the mean
    loss = tf.reduce_sum(tf.maximum(basic_loss, 0.0))
    
    return loss


In [4]:
import numpy as np
import tensorflow as tf

def img_to_encoding(image_path, Dmodel):
    # Load and preprocess the image
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(229, 229)) 
    img = np.around(np.array(img) / 255.0, decimals=12)  


    x_train = np.expand_dims(img, axis=0)  

    # Predict the embedding
    embedding = Dmodel.predict_on_batch(x_train)  # Get the embedding from the model

    # Squeeze to remove any single-dimensional entries
    embedding = np.squeeze(embedding)

    # Normalize the embedding to get a unit vector
    return embedding / np.linalg.norm(embedding, ord=2)  # Normalize to unit vector


In [5]:
triplets = {
    "nasir garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p1\anchor.jpg",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p1\positve.png",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p1\negative.jpg"
    },
    "muhammad garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p2\anchor.jpg",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p2\positive.jpg",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p2\negative.jpg"
    },
     "ahmad garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p3\anchor.png",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p3\positive.jpg",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p3\negative.jpg"
    },
     "amina garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p4\anchor.jpg",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p4\positive.jpg",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\p4\negative.jpg"
    },
     "hamida garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn5\anchor.png",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn5\positive.png",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn5\negative.jpg"
    },
     "baby": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn6\anchor.jpg",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn6\positive.jpg",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn6\negative.jpg"
    },
     "hauwa garba": {
        "anchor": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn7\anchor.png",
        "positive": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn7\positive.jpeg",
        "negative": r"C:\Users\Garba Buhari\Desktop\face recognition CNN\muhammad faces\pn7\negative.jpg"
    },
}
encodings = {}
for person, images in triplets.items():
    encodings[person] = {
        "anchor": img_to_encoding(images['anchor'], Dmodel),
        "positive": img_to_encoding(images['positive'], Dmodel),
        "negative": img_to_encoding(images['negative'], Dmodel)
    }

In [22]:
def verify(image_path, person, triplets, Dmodel): 
    # Check if the person exists in the triplet
    if person not in triplets:
        print(f"Person '{person}' not found in the database.")
        return None, False
    
    try:
        # Compute the encoding for the provided image
        encoding = img_to_encoding(image_path, Dmodel)
        
        # Compute distance using the anchor image of the specified person
        person_encoding = img_to_encoding(triplets[person]['anchor'], Dmodel)
        dist = np.linalg.norm(encoding - person_encoding)  # Calculate distance with anchor
        
        # Determine if the door should open based on the distance threshold
        door_open = dist < 0.5  # Adjust this threshold as necessary
        
        # Provide feedback based on verification result
        if door_open:
            print(f"It's {person}, welcome in!")
        else:
            print(f"It's not {person}, please go away.")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None, False
    
    return dist, door_open

# Example usage
img_path = r"C:\Users\Garba Buhari\Desktop\face recognition CNN\Screenshot_20200908-222228.png"
person = "muhammad garba"
dist, door_open = verify(img_path, person, triplets, Dmodel)
print(f"distance:{dist}")

It's not muhammad garba, please go away.
distance:0.7233600616455078
